# 🌊 Experimentos Deep Learning - Wavelets Fixas (db2)

## Objetivo
Avaliar modelos de Deep Learning usando coeficientes wavelet fixos (db2) como pré-processamento:
- **CNN + Wavelet db2**
- **LSTM + Wavelet db2**
- **CNN-LSTM + Wavelet db2**
- **Transformer + Wavelet db2**

## Pipeline
1. Carregar dados
2. Extrair coeficientes wavelet multi-nível (como canais)
3. Treinar cada modelo
4. Comparar com resultados do sinal raw

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

# TensorFlow
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponível: {tf.config.list_physical_devices('GPU')}")

# Imports locais
import sys
sys.path.append('.')
from src.models import (
    create_cnn_model, create_lstm_model, 
    create_cnn_lstm_model, create_transformer_model,
    get_callbacks
)
from src.feature_extraction import WaveletFeatureExtractor
from src.evaluation import RegressionEvaluator, ResultsManager
from src.visualization import ExperimentVisualizer
from config.experiment_config import (
    DATA_DIR, RESULTS_DIR, MODELS_DIR,
    DL_TRAINING_CONFIG, DL_MODELS_CONFIG, WAVELET_CONFIG
)

# Configuração
plt.style.use('seaborn-v0_8-whitegrid')
(RESULTS_DIR / "dl_wavelet_experiments").mkdir(parents=True, exist_ok=True)

print("\n✅ Imports realizados com sucesso!")

## 1. Carregar e Transformar Dados

In [ ]:
# Carregar datasets (raw)
X_train_raw = np.load(DATA_DIR / "X_train.npy")
y_train = np.load(DATA_DIR / "y_train.npy")
X_val_raw = np.load(DATA_DIR / "X_val.npy")
y_val = np.load(DATA_DIR / "y_val.npy")
X_test_raw = np.load(DATA_DIR / "X_test.npy")
y_test = np.load(DATA_DIR / "y_test.npy")

print(f"📦 Dados Carregados (Raw):")
print(f"  Train: {X_train_raw.shape}")
print(f"  Val:   {X_val_raw.shape}")
print(f"  Test:  {X_test_raw.shape}")

In [ ]:
# Configurar extrator de wavelet
wavelet_extractor = WaveletFeatureExtractor(
    wavelet=WAVELET_CONFIG['wavelet_type'],
    level=WAVELET_CONFIG['decomposition_level'],
    mode=WAVELET_CONFIG['mode']
)

print(f"Extrator Wavelet:")
print(f"  Wavelet: {WAVELET_CONFIG['wavelet_type']}")
print(f"  Níveis: {WAVELET_CONFIG['decomposition_level']}")

# Transformar para coeficientes multi-nível (como canais)
print("\nTransformando sinais para coeficientes wavelet...")
t0 = time.time()

X_train = wavelet_extractor.get_multilevel_coefficients(X_train_raw, align_length=True)
X_val = wavelet_extractor.get_multilevel_coefficients(X_val_raw, align_length=True)
X_test = wavelet_extractor.get_multilevel_coefficients(X_test_raw, align_length=True)

print(f"  Tempo: {time.time()-t0:.2f}s")

print(f"\n📊 Dados Transformados (Wavelet Coefficients):")
print(f"  Train: {X_train.shape}  [samples, length, channels=níveis+1]")
print(f"  Val:   {X_val.shape}")
print(f"  Test:  {X_test.shape}")
print(f"\n  Canais: [Approx, Detail_1, Detail_2, Detail_3, Detail_4]")

input_shape = X_train.shape[1:]
print(f"\nInput shape para modelos: {input_shape}")

## 2. Visualização dos Coeficientes Wavelet

In [ ]:
# Visualizar uma amostra transformada
sample_idx = 1000

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Sinal original
axes[0, 0].plot(X_train_raw[sample_idx], alpha=0.7)
axes[0, 0].set_title('Sinal Original (Raw)')
axes[0, 0].grid(True, alpha=0.3)

# Coeficientes wavelet
channel_names = ['Approx (A4)', 'Detail 1 (D1)', 'Detail 2 (D2)', 
                 'Detail 3 (D3)', 'Detail 4 (D4)']
for i, (ax, name) in enumerate(zip(axes.flatten()[1:], channel_names)):
    ax.plot(X_train[sample_idx, :, i], alpha=0.7)
    ax.set_title(f'Canal {i}: {name}')
    ax.grid(True, alpha=0.3)

plt.suptitle(f'Transformação Wavelet (db2) - Amostra {sample_idx}', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(RESULTS_DIR / "dl_wavelet_experiments" / "wavelet_transformation_sample.png", dpi=150)
plt.show()

## 3. Configuração

In [ ]:
# Gerenciadores
results_manager = ResultsManager(RESULTS_DIR / "dl_wavelet_experiments")
evaluator = RegressionEvaluator()
visualizer = ExperimentVisualizer()

training_config = DL_TRAINING_CONFIG.copy()

# Armazenar resultados
all_results = {}
all_histories = {}

## 4. Experimento 1: CNN + Wavelet

In [ ]:
print("="*70)
print("🔵 Experimento: CNN com Wavelet db2")
print("="*70)

tf.keras.backend.clear_session()

# Criar modelo
cnn_params = DL_MODELS_CONFIG['CNN']
model_cnn = create_cnn_model(input_shape, params=cnn_params)
model_cnn.summary()

# Callbacks
model_path = str(MODELS_DIR / "wavelet_cnn_best.keras")
callbacks = get_callbacks(model_path, patience_early=15, patience_lr=7)

# Treinar
t0 = time.time()
history_cnn = model_cnn.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=training_config['epochs'],
    batch_size=training_config['batch_size'],
    callbacks=callbacks,
    verbose=1
)
elapsed = time.time() - t0

# Predições
y_pred_cnn = model_cnn.predict(X_test, verbose=0).flatten()

# Métricas
cnn_metrics = evaluator.evaluate(y_test, y_pred_cnn)

print(f"\n📊 Resultados CNN (Wavelet db2):")
print(f"  RMSE: {cnn_metrics['rmse']:.6f}")
print(f"  MAE:  {cnn_metrics['mae']:.6f}")
print(f"  R²:   {cnn_metrics['r2']:.6f}")
print(f"  Tempo: {elapsed:.2f}s")

all_results['Wavelet_CNN'] = {
    'metrics': cnn_metrics,
    'time': elapsed,
    'epochs': len(history_cnn.history['loss']),
    'y_pred': y_pred_cnn,
    'params': model_cnn.count_params()
}
all_histories['Wavelet_CNN'] = history_cnn.history

results_manager.log_experiment(
    'DL_Wavelet', 'CNN', cnn_metrics,
    {'params': cnn_params, 'wavelet': WAVELET_CONFIG}
)

## 5. Experimento 2: LSTM + Wavelet

In [ ]:
print("="*70)
print("🔵 Experimento: LSTM com Wavelet db2")
print("="*70)

tf.keras.backend.clear_session()

# Criar modelo
lstm_params = DL_MODELS_CONFIG['LSTM']
model_lstm = create_lstm_model(input_shape, params=lstm_params)
model_lstm.summary()

# Callbacks
model_path = str(MODELS_DIR / "wavelet_lstm_best.keras")
callbacks = get_callbacks(model_path, patience_early=15, patience_lr=7)

# Treinar
t0 = time.time()
history_lstm = model_lstm.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=training_config['epochs'],
    batch_size=training_config['batch_size'],
    callbacks=callbacks,
    verbose=1
)
elapsed = time.time() - t0

# Predições
y_pred_lstm = model_lstm.predict(X_test, verbose=0).flatten()

# Métricas
lstm_metrics = evaluator.evaluate(y_test, y_pred_lstm)

print(f"\n📊 Resultados LSTM (Wavelet db2):")
print(f"  RMSE: {lstm_metrics['rmse']:.6f}")
print(f"  MAE:  {lstm_metrics['mae']:.6f}")
print(f"  R²:   {lstm_metrics['r2']:.6f}")

all_results['Wavelet_LSTM'] = {
    'metrics': lstm_metrics,
    'time': elapsed,
    'epochs': len(history_lstm.history['loss']),
    'y_pred': y_pred_lstm,
    'params': model_lstm.count_params()
}
all_histories['Wavelet_LSTM'] = history_lstm.history

results_manager.log_experiment(
    'DL_Wavelet', 'LSTM', lstm_metrics,
    {'params': lstm_params, 'wavelet': WAVELET_CONFIG}
)

## 6. Experimento 3: CNN-LSTM + Wavelet

In [ ]:
print("="*70)
print("🔵 Experimento: CNN-LSTM com Wavelet db2")
print("="*70)

tf.keras.backend.clear_session()

# Criar modelo
cnn_lstm_params = DL_MODELS_CONFIG['CNN_LSTM']
model_cnn_lstm = create_cnn_lstm_model(input_shape, params=cnn_lstm_params)
model_cnn_lstm.summary()

# Callbacks
model_path = str(MODELS_DIR / "wavelet_cnn_lstm_best.keras")
callbacks = get_callbacks(model_path, patience_early=15, patience_lr=7)

# Treinar
t0 = time.time()
history_cnn_lstm = model_cnn_lstm.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=training_config['epochs'],
    batch_size=training_config['batch_size'],
    callbacks=callbacks,
    verbose=1
)
elapsed = time.time() - t0

# Predições
y_pred_cnn_lstm = model_cnn_lstm.predict(X_test, verbose=0).flatten()

# Métricas
cnn_lstm_metrics = evaluator.evaluate(y_test, y_pred_cnn_lstm)

print(f"\n📊 Resultados CNN-LSTM (Wavelet db2):")
print(f"  RMSE: {cnn_lstm_metrics['rmse']:.6f}")
print(f"  MAE:  {cnn_lstm_metrics['mae']:.6f}")
print(f"  R²:   {cnn_lstm_metrics['r2']:.6f}")

all_results['Wavelet_CNN_LSTM'] = {
    'metrics': cnn_lstm_metrics,
    'time': elapsed,
    'epochs': len(history_cnn_lstm.history['loss']),
    'y_pred': y_pred_cnn_lstm,
    'params': model_cnn_lstm.count_params()
}
all_histories['Wavelet_CNN_LSTM'] = history_cnn_lstm.history

results_manager.log_experiment(
    'DL_Wavelet', 'CNN_LSTM', cnn_lstm_metrics,
    {'params': cnn_lstm_params, 'wavelet': WAVELET_CONFIG}
)

## 7. Experimento 4: Transformer + Wavelet

In [ ]:
print("="*70)
print("🔵 Experimento: Transformer com Wavelet db2")
print("="*70)

tf.keras.backend.clear_session()

# Criar modelo
transformer_params = DL_MODELS_CONFIG['Transformer']
model_transformer = create_transformer_model(input_shape, params=transformer_params)
model_transformer.summary()

# Callbacks
model_path = str(MODELS_DIR / "wavelet_transformer_best.keras")
callbacks = get_callbacks(model_path, patience_early=15, patience_lr=7)

# Treinar
t0 = time.time()
history_transformer = model_transformer.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=training_config['epochs'],
    batch_size=training_config['batch_size'],
    callbacks=callbacks,
    verbose=1
)
elapsed = time.time() - t0

# Predições
y_pred_transformer = model_transformer.predict(X_test, verbose=0).flatten()

# Métricas
transformer_metrics = evaluator.evaluate(y_test, y_pred_transformer)

print(f"\n📊 Resultados Transformer (Wavelet db2):")
print(f"  RMSE: {transformer_metrics['rmse']:.6f}")
print(f"  MAE:  {transformer_metrics['mae']:.6f}")
print(f"  R²:   {transformer_metrics['r2']:.6f}")

all_results['Wavelet_Transformer'] = {
    'metrics': transformer_metrics,
    'time': elapsed,
    'epochs': len(history_transformer.history['loss']),
    'y_pred': y_pred_transformer,
    'params': model_transformer.count_params()
}
all_histories['Wavelet_Transformer'] = history_transformer.history

results_manager.log_experiment(
    'DL_Wavelet', 'Transformer', transformer_metrics,
    {'params': transformer_params, 'wavelet': WAVELET_CONFIG}
)

## 8. Comparação dos Resultados

In [ ]:
# Criar DataFrame comparativo
comparison_data = []
for model_name, result in all_results.items():
    row = {
        'Model': model_name,
        'RMSE': result['metrics']['rmse'],
        'MAE': result['metrics']['mae'],
        'R²': result['metrics']['r2'],
        'Params': result['params'],
        'Time (s)': result['time'],
        'Epochs': result['epochs']
    }
    comparison_data.append(row)

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('RMSE')

print("\n" + "="*70)
print("📊 COMPARAÇÃO FINAL - Deep Learning com Wavelet db2")
print("="*70)
print(comparison_df.to_string(index=False))

# Salvar
comparison_df.to_csv(RESULTS_DIR / "dl_wavelet_experiments" / "comparison_dl_wavelet.csv", index=False)

In [ ]:
# Visualização comparativa
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

metrics_to_plot = ['RMSE', 'MAE', 'R²']
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(comparison_df)))

for idx, metric in enumerate(metrics_to_plot):
    data = comparison_df.set_index('Model')[metric].sort_values(
        ascending=(metric != 'R²')
    )
    bars = axes[idx].barh(data.index, data.values, color=colors)
    axes[idx].set_xlabel(metric)
    axes[idx].set_title(f'Comparação: {metric}')
    axes[idx].grid(True, alpha=0.3, axis='x')
    
    for bar, val in zip(bars, data.values):
        axes[idx].text(val, bar.get_y() + bar.get_height()/2,
                      f'{val:.4f}', va='center', ha='left', fontsize=9)

plt.suptitle('Deep Learning com Wavelet db2', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(RESULTS_DIR / "dl_wavelet_experiments" / "comparison_dl_wavelet.png", dpi=150, bbox_inches='tight')
plt.show()

## 9. Resumo

In [ ]:
# Encontrar melhor modelo
best_model_name = comparison_df.iloc[0]['Model']

print("\n" + "="*70)
print("📋 RESUMO - Experimentos DL com Wavelet db2")
print("="*70)
print(f"\n✅ Modelos avaliados: {len(all_results)}")
print(f"✅ Melhor modelo: {best_model_name}")
print(f"✅ Melhor RMSE: {comparison_df.iloc[0]['RMSE']:.6f}")
print(f"✅ Melhor R²: {comparison_df.iloc[0]['R²']:.6f}")
print(f"\n📁 Resultados salvos em: {RESULTS_DIR / 'dl_wavelet_experiments'}")
print("\n🎉 Notebook concluído com sucesso!")